# Create clean CHARTEVENTS data set

Purpose of this script is to prepare the MIMIC-III data set `CHARTEVENTS.csv` for later use in the master project context by cleaning the vital parameter values and alarm threshold values it contains.

This cleaning includes filtering the data set to a subset that includes only the relevant CHARTEVENT rows with sufficient measurements. In addition, both vital parameter and alarm threshold values that are outside the respective clinically valid value ranges are removed (e.g. a heart rate of 1000 bpm and a heart rate alarm threshold of type *high* of 4000 bpm). Furthermore, two more complex cleanup steps are performed for alarm threshold values: *Revert local threshold swap* and *Remove threshold values due to local overlap*. These steps aim to correct implausible alarm threshold value series. To be precise, those sections of the alarm threshold value series are corrected where the threshold value of type *high* is lower than the threshold value of type *low* and vice versa. According to the medical experts consulted, such an overlap of alarm threshold values makes no sense.

**Revert local threshold swap**: In some cases, the thresholds appear to be exactly swapped, i.e. the threshold of type *high* appears to be actually the threshold of type *low* and vice versa. Such an apparent swap is usually not present over the entire period of the alarm threshold value series, but only at certain sections, i.e. locations. Accordingly, we call it a *local threshold swap*. Local threshold swaps are corrected by reversing the swap, i.e. by swapping the alarm threshold value of type *high* with that of type *low* at the affected locations.

**Remove threshold values due to local overlap**: There are also cases where the alarm threshold value series of type *high* and *low* overlap, but are not exactly swapped (e.g. the alarm threshold value of the type *high* is set unreasonably low and falls below the alarm threshold value of type *low*, while the latter seems to have a correct value). Such an overlap is usually not present over the entire period of the alarm threshold value series, but only at certain sections, i.e. locations. Accordingly, we call it a *local overlap*. These cases cannot be corrected by swapping. Also, it is not possible (without a lot of effort) to determine whether only one of the two thresholds is unreasonably high/low or both. Therefore, the threshold values of both types (high and low) are removed at those locations where they overlap. Hence, we call this step *Remove threshold values due to local overlap*.

As the example of *Revert local threshold swap* and *Remove threshold values due to local overlap* illustrates, the **order of the cleaning steps matters**. For example, *Revert local threshold swaps* must be done first, then *Remove threshold values due to local overlap*. Otherwise, exact swaps would no longer be identifiable because they would already have been removed as a local overlap.

The following overview shows the cleaning steps in sequence:

1. Create chartevents_subset
    - Created file: `chartevents_subset.parquet` (subset of CHARTEVENTS data frame)
2. Remove rows with insufficient values from chartevents_subset
    - Created file: `chartevents_cleaning_02_sufficient_values_only.parquet` (modified CHARTEVENTS data frame)
3. Remove measurement values outside clinically valid ranges
    - 3.1 Flag measurements outside clinically valid ranges
    - 3.2 Introduce new VALUENUM_CLEAN column and set measurement values outside valid ranges to NaN
    - Created file: `chartevents_cleaning_03_measurements_in_valid_range.parquet` (modified CHARTEVENTS data frame)
4. Revert local threshold swap
    - 4.1 Identify candidates for local threshold swap
    - 4.2 Prepare data set for local threshold swap
    - 4.3 Perform local threshold swap
    - Created file: `chartevents_cleaning_04_exact_threshold_swaps_reverted.parquet` (modified CHARTEVENTS data frame)
5. Remove alarm threshold values outside clinically valid ranges
    - Created file: `chartevents_cleaning_05_thresholds_in_valid_range.parquet` (modified CHARTEVENTS data frame)
6. Remove threshold values due to local overlap
    - 6.1 Identify candidates for threshold removal due to local overlap
    - 6.2 Prepare data set for threshold removal due to local overlap
    - 6.3 Perform threshold removal due to local overlap
    - Created file: `chartevents_cleaning_06_overlapping_thresholds_removed.parquet` (modified CHARTEVENTS data frame), which is the final data frame `chartevents_clean.parquet` as of the master project completion date. 

As the overview of the steps shows, auxiliary data frames as well as intermediate results of the CHARTEVENTS cleanup are saved as parquet files. This was done to avoid recomputation of data frames when working on individual cleaning sections with limited computing resources.

## 1. Create chartevents_subset 

- Create subset of the MIMIC-III data set called `CHARTEVENTS.csv` (see also respective [MIMIC schema website](https://mit-lcp.github.io/mimic-schema-spy/tables/chartevents.html))
- No change in columns, keep all of them
- Reduce number of rows by:
  - Filtering for relevant ITEMIDs,
  - Removing rows without ICUSTAY_ID, and
  - Removing rows flagged as erroneous

In [1]:
import pandas as pd
import dask.dataframe as dd
import pyarrow as pa
from dask.diagnostics import ProgressBar

path_to_dir_chartevents_csv = '../data/mimic/'
path_to_dir_chartevents_clean = './reportdata/'

# Read CHARTEVENTS.csv as Dask DataFrame
# Data types based on MIMIC schema specification https://mit-lcp.github.io/mimic-schema-spy/tables/chartevents.html
# Problem: Complicated use of intger data types with NaNs in Pandas, see https://pandas.pydata.org/pandas-docs/stable/user_guide/gotchas.html#nan-integer-na-values-and-na-type-promotions
# Decision: Floats and integers are read in as 'float64', strings as 'object', and timestamps via Dask's parse_dates provided for this purpose.
chartevents = dd.read_csv(path_to_dir_chartevents_csv+'CHARTEVENTS.csv', parse_dates=['CHARTTIME','STORETIME'], dtype={
    'ROW_ID': 'float64', # int4 according to specification
    'SUBJECT_ID': 'float64', # int4 according to specification
    'HADM_ID': 'float64', # int4 according to specification
    'ICUSTAY_ID': 'float64', # int4 according to specification
    'ITEMID': 'float64', # int4 according to specification
    'CGID': 'float64', # int4 according to specification
    'VALUE': 'object',
    'VALUENUM': 'float64', # float8 according to specification
    'VALUEUOM': 'object',
    'WARNING': 'float64', # int4 according to specification
    'ERROR': 'float64', # int4 according to specification
    'RESULTSTATUS': 'object',
    'STOPPED': 'object'})

# Create list of relevant ITEMIDs to filter by
itemid_filter = [220045, 220046, 220047, 220179, 223751, 223752, 220277, 223769, 223770]
# 220045 Heart Rate
# 220046 Heart rate Alarm - High
# 220047 Heart Rate Alarm - Low
# 220179 Non Invasive Blood Pressure systolic
# 223751 Non-Invasive Blood Pressure Alarm - High
# 223752 Non-Invasive Blood Pressure Alarm - Low
# 220277 O2 saturation pulseoxymetry
# 223769 O2 Saturation Pulseoxymetry Alarm - High
# 223770 O2 Saturation Pulseoxymetry Alarm - Low

with ProgressBar():
    # Filter by ITEMIDs
    chartevents_subset = chartevents[chartevents.ITEMID.isin(itemid_filter)]
    # Drop rows without ICUSTAY_ID
    chartevents_subset = chartevents_subset.dropna(how='any', subset=['ICUSTAY_ID'])
    # Keep only rows without error, coded by value 0 in ERROR column
    chartevents_subset = chartevents_subset[chartevents_subset.ERROR.isin([0])]
    # Apply previously defined commands to Dask DataFrame, resulting in desired Pandas DataFrame
    chartevents_subset = chartevents_subset.compute()

# Sort rows and reset index (not essential, but provides better overview)
chartevents_subset = chartevents_subset.sort_values(by=['ICUSTAY_ID', 'CHARTTIME','ITEMID']).reset_index(drop=True)

# Save as parquet file
chartevents_subset.to_parquet(path_to_dir_chartevents_clean+'chartevents_subset.parquet', engine='pyarrow')

[########################################] | 100% Completed | 17min 15.4s


## 2. Remove rows with insufficient values from chartevents_subset

The CHARTEVENTS subset contains ...
- ICU stays that have *threshold* values but no *measurement* values for certain parameters.
- ICU stays that have only *measurement* values but no *threshold* values for certain parameters.
- ICU stays that have only a single measurement value for certain parameters.

Therefore, remove those ICU stay parameter combinations ...
- Without measurement values,
- Without threshold values, and
- With only one measurement value.

In [1]:
import pandas as pd
import pyarrow as pa
path_to_dir_chartevents_clean = './reportdata/'
# Read chartevents_subset from parquet file
chartevents_subset = pd.read_parquet(path_to_dir_chartevents_clean+'chartevents_subset.parquet', engine='pyarrow')

In [2]:
# Create data frame containing a list of the available ITEMIDs for each ICU stay
itemids_per_icustay = chartevents_subset.groupby(['ICUSTAY_ID'])['ITEMID'].unique().apply(list).reset_index()
itemids_per_icustay['ITEMID'] = itemids_per_icustay.ITEMID.apply(lambda x: sorted(x))
display(itemids_per_icustay)

,ICUSTAY_ID,ITEMID
0,200001.0,"[220045.0, 220046.0, 220047.0, 220179.0, 22027..."
1,200010.0,"[220045.0, 220046.0, 220047.0, 220179.0, 22027..."
2,200011.0,"[220045.0, 220046.0, 220047.0, 220179.0, 22027..."
3,200016.0,"[220045.0, 220046.0, 220047.0, 220179.0, 22027..."
4,200021.0,"[220045.0, 220046.0, 220047.0, 220179.0, 22027..."
...,...,...
23441,299956.0,"[220045.0, 220046.0, 220047.0, 220179.0, 22027..."
23442,299957.0,"[220045.0, 220046.0, 220047.0, 220179.0, 22027..."
23443,299962.0,"[220045.0, 220046.0, 220047.0, 220179.0, 22027..."
23444,299979.0,"[220045.0, 220046.0, 220047.0, 220179.0, 22027..."


In [3]:
# Identify ICU stays that have thresholds but no measurements
icustayids_without_measurement_hr = pd.Series(dtype='float64')
icustayids_without_measurement_bp = pd.Series(dtype='float64')
icustayids_without_measurement_o2 = pd.Series(dtype='float64')

for i, icustay in itemids_per_icustay.iterrows():
    if (([220046] in icustay.ITEMID) and ([220047] in icustay.ITEMID) and ([220045] not in icustay.ITEMID)):
        icustayids_without_measurement_hr = icustayids_without_measurement_hr.append(pd.Series(icustay.ICUSTAY_ID)).reset_index(drop=True)
    
    if (([223751] in icustay.ITEMID) and ([223752] in icustay.ITEMID) and ([220179] not in icustay.ITEMID)):
        icustayids_without_measurement_bp = icustayids_without_measurement_bp.append(pd.Series(icustay.ICUSTAY_ID)).reset_index(drop=True)
    
    if (([223769] in icustay.ITEMID) and ([223770] in icustay.ITEMID) and ([220277] not in icustay.ITEMID)):
        icustayids_without_measurement_o2 = icustayids_without_measurement_o2.append(pd.Series(icustay.ICUSTAY_ID)).reset_index(drop=True)

# Print some statistics
print("Number of ICU stays that have thresholds but no measurements:")
print("  HR:   ",len(icustayids_without_measurement_hr))
print("  NBPs: ",len(icustayids_without_measurement_bp))
print("  SpO2: ",len(icustayids_without_measurement_o2))
print("  ______________")
print("  Sum:  ",len(icustayids_without_measurement_hr) + len(icustayids_without_measurement_bp) + len(icustayids_without_measurement_o2))

Number of ICU stays that have thresholds but no measurements:
  HR:    6
  NBPs:  135
  SpO2:  20
  ______________
  Sum:   161


In [9]:
# Identify ICU stays that have measurements but at least one missing threshold series
icustayids_with_missing_threshold_hr = pd.Series(dtype='float64')
icustayids_with_missing_threshold_bp = pd.Series(dtype='float64')
icustayids_with_missing_threshold_o2 = pd.Series(dtype='float64')

for i, icustay in itemids_per_icustay.iterrows():
    if ([220045] in icustay.ITEMID) and (([220046] not in icustay.ITEMID) or ([220047] not in icustay.ITEMID)):
        icustayids_with_missing_threshold_hr = icustayids_with_missing_threshold_hr.append(pd.Series(icustay.ICUSTAY_ID)).reset_index(drop=True)
    
    if ([220179] in icustay.ITEMID) and (([223751] not in icustay.ITEMID) or ([223752] not in icustay.ITEMID)):
        icustayids_with_missing_threshold_bp = icustayids_with_missing_threshold_bp.append(pd.Series(icustay.ICUSTAY_ID)).reset_index(drop=True)
    
    if ([220277] in icustay.ITEMID) and (([223769] not in icustay.ITEMID) or ([223770] not in icustay.ITEMID)):
        icustayids_with_missing_threshold_o2 = icustayids_with_missing_threshold_o2.append(pd.Series(icustay.ICUSTAY_ID)).reset_index(drop=True)

# Print some statistics
print("Number of ICU stays with at least one missing threshold series:")
print("  HR:   ",len(icustayids_with_missing_threshold_hr))
print("  NBPs: ",len(icustayids_with_missing_threshold_bp))
print("  SpO2: ",len(icustayids_with_missing_threshold_o2))
print("  ______________")
print("  Sum:  ",len(icustayids_with_missing_threshold_hr) + len(icustayids_with_missing_threshold_bp) + len(icustayids_with_missing_threshold_o2))

Number of ICU stays with at least one missing threshold series:
  HR:    61
  NBPs:  879
  SpO2:  63
  ______________
  Sum:   1003


In [10]:
# Create a data frame, which will be used as filter.
# This data frame contains all ICU stay parameter combinations to be removed due to missing measurement or threshold values.
# This data frame must consist of the two columns ICUSTAY_ID and ITEMID.

# For all ICU stays that have been identified as having mising measurement or threshold values ...
icustayids_hr = pd.concat([icustayids_without_measurement_hr, icustayids_with_missing_threshold_hr]).unique()
icustayids_bp = pd.concat([icustayids_without_measurement_bp, icustayids_with_missing_threshold_bp]).unique()
icustayids_o2 = pd.concat([icustayids_without_measurement_o2, icustayids_with_missing_threshold_o2]).unique()
# ... generate combinations with the three ITEMIDs of the respective parameter (measurement, threshold high, and threshold low)
missing_values_filter = pd.concat([
    pd.DataFrame({'ICUSTAY_ID': icustayids_hr, 'ITEMID': 220045}),
    pd.DataFrame({'ICUSTAY_ID': icustayids_hr, 'ITEMID': 220046}),
    pd.DataFrame({'ICUSTAY_ID': icustayids_hr, 'ITEMID': 220047}),
    pd.DataFrame({'ICUSTAY_ID': icustayids_bp, 'ITEMID': 220179}),
    pd.DataFrame({'ICUSTAY_ID': icustayids_bp, 'ITEMID': 223751}),
    pd.DataFrame({'ICUSTAY_ID': icustayids_bp, 'ITEMID': 223752}),
    pd.DataFrame({'ICUSTAY_ID': icustayids_o2, 'ITEMID': 220277}),
    pd.DataFrame({'ICUSTAY_ID': icustayids_o2, 'ITEMID': 223769}),
    pd.DataFrame({'ICUSTAY_ID': icustayids_o2, 'ITEMID': 223770})
    ])

# Print some statistics
print("Number of ICU stay parameter combinations to be removed due to missing measurement or threshold values:")
print("  HR:   ",len(icustayids_hr))
print("  NBPs: ",len(icustayids_bp))
print("  SpO2: ",len(icustayids_o2))
print("  ______________")
print("  Sum:  ",len(icustayids_hr) + len(icustayids_bp) + len(icustayids_o2))
# Number of ICUSTAY_ID ITEMID comninations in filter is three times the ICU stay parameter combinations, since 3 ITEMIDs apply per parameter (measurement, threshold high, and threshold low).
display(missing_values_filter)

Number of ICU stay parameter combinations to be removed due to missing measurement or threshold values:
  HR:    67
  NBPs:  1014
  SpO2:  83
  ______________
  Sum:   1164


,ICUSTAY_ID,ITEMID
0,214738.0,220045
1,229737.0,220045
2,239953.0,220045
3,244456.0,220045
4,261030.0,220045
...,...,...
78,294198.0,223770
79,296917.0,223770
80,298497.0,223770
81,298841.0,223770


In [11]:
# Identify ICU stays that have only a single measurement value
# Create data frame with number of measurement values per ICU stay and measurement ITEMID
n_measurements_per_icustay_and_itemid = chartevents_subset[chartevents_subset.ITEMID.isin([220045, 220179, 220277])][['ICUSTAY_ID','ITEMID','VALUENUM']]
n_measurements_per_icustay_and_itemid = n_measurements_per_icustay_and_itemid.groupby(['ICUSTAY_ID','ITEMID']).count()
n_measurements_per_icustay_and_itemid = n_measurements_per_icustay_and_itemid.rename(columns = {'VALUENUM':'VALUENUM_COUNT'}).reset_index()

# Select ICU stays that have only a single measurement value ...
icustayids_hr = n_measurements_per_icustay_and_itemid[(n_measurements_per_icustay_and_itemid.ITEMID == 220045) & (n_measurements_per_icustay_and_itemid.VALUENUM_COUNT == 1)]['ICUSTAY_ID']
icustayids_bp = n_measurements_per_icustay_and_itemid[(n_measurements_per_icustay_and_itemid.ITEMID == 220179) & (n_measurements_per_icustay_and_itemid.VALUENUM_COUNT == 1)]['ICUSTAY_ID']
icustayids_o2 = n_measurements_per_icustay_and_itemid[(n_measurements_per_icustay_and_itemid.ITEMID == 220277) & (n_measurements_per_icustay_and_itemid.VALUENUM_COUNT == 1)]['ICUSTAY_ID']
# ... and generate combinations with the three ITEMIDs of the respective parameter (measurement, threshold high, and threshold low)
single_measurement_filter = pd.concat([
    pd.DataFrame({'ICUSTAY_ID': icustayids_hr, 'ITEMID': 220045}),
    pd.DataFrame({'ICUSTAY_ID': icustayids_hr, 'ITEMID': 220046}),
    pd.DataFrame({'ICUSTAY_ID': icustayids_hr, 'ITEMID': 220047}),
    pd.DataFrame({'ICUSTAY_ID': icustayids_bp, 'ITEMID': 220179}),
    pd.DataFrame({'ICUSTAY_ID': icustayids_bp, 'ITEMID': 223751}),
    pd.DataFrame({'ICUSTAY_ID': icustayids_bp, 'ITEMID': 223752}),
    pd.DataFrame({'ICUSTAY_ID': icustayids_o2, 'ITEMID': 220277}),
    pd.DataFrame({'ICUSTAY_ID': icustayids_o2, 'ITEMID': 223769}),
    pd.DataFrame({'ICUSTAY_ID': icustayids_o2, 'ITEMID': 223770})
    ])

# Print some statistics
print("Number of ICU stay parameter combinations to be removed due to single measurement:")
print("  HR:   ",len(icustayids_hr))
print("  NBPs: ",len(icustayids_bp))
print("  SpO2: ",len(icustayids_o2))
print("  ______________")
print("  Sum:  ",len(icustayids_hr) + len(icustayids_bp) + len(icustayids_o2))

Number of ICU stay parameter combinations to be removed due to single measurement:
  HR:    15
  NBPs:  179
  SpO2:  24
  ______________
  Sum:   218


In [12]:
# Merge filter data frames and drop duplicates
# Duplicates occur when both filters apply, e.g. when an ICU stay has a single heart rate measurement and a missing threshold series.
insufficient_values_filter = missing_values_filter.append(single_measurement_filter).drop_duplicates()

# Print some statistics
print("Number of ICU stay parameter combinations to be removed due to insufficient values:")
print("  HR:   ",len(insufficient_values_filter[insufficient_values_filter.ITEMID == 220045]))
print("  NBPs: ",len(insufficient_values_filter[insufficient_values_filter.ITEMID == 220179]))
print("  SpO2: ",len(insufficient_values_filter[insufficient_values_filter.ITEMID == 220277]))
print("  ______________")
print("  Sum:  ",len(insufficient_values_filter)//3)

Number of ICU stay parameter combinations to be removed due to insufficient values:
  HR:    75
  NBPs:  1122
  SpO2:  96
  ______________
  Sum:   1293


In [13]:
# Filter the chartevents_subset based on insufficient_values_filter
df = pd.merge(chartevents_subset, insufficient_values_filter, how='outer', left_on=['ICUSTAY_ID', 'ITEMID'], right_on=['ICUSTAY_ID', 'ITEMID'], indicator=True)
# Keep only those chartevents_subset rows where the ICUSTAY_ID ITEMID combination is not present in insufficient_values_filter
df = df[df._merge == 'left_only'].drop('_merge', 1)
chartevents_cleaning_02_sufficient_values_only = df
display(chartevents_cleaning_02_sufficient_values_only)

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
0,14005075.0,55973.0,152234.0,200001.0,220045.0,2181-11-25 19:06:00,2181-11-25 19:17:00,20622.0,115,115.0,bpm,0.0,0.0,None,None
1,14005090.0,55973.0,152234.0,200001.0,220045.0,2181-11-25 19:16:00,2181-11-25 19:16:00,20622.0,114,114.0,bpm,0.0,0.0,None,None
2,14005105.0,55973.0,152234.0,200001.0,220045.0,2181-11-25 20:00:00,2181-11-25 22:02:00,21108.0,113,113.0,bpm,0.0,0.0,None,None
3,14005111.0,55973.0,152234.0,200001.0,220045.0,2181-11-25 21:00:00,2181-11-25 22:02:00,21108.0,108,108.0,bpm,0.0,0.0,None,None
4,14005117.0,55973.0,152234.0,200001.0,220045.0,2181-11-25 22:00:00,2181-11-25 22:02:00,21108.0,110,110.0,bpm,0.0,0.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7886595,20563805.0,69587.0,158288.0,299998.0,223751.0,2181-07-06 20:19:00,2181-07-06 20:19:00,20053.0,160,160.0,mmHg,0.0,0.0,None,None
7886596,20563920.0,69587.0,158288.0,299998.0,223751.0,2181-07-07 08:00:00,2181-07-07 10:04:00,18462.0,160,160.0,mmHg,0.0,0.0,None,None
7886597,20563683.0,69587.0,158288.0,299998.0,223752.0,2181-07-06 08:02:00,2181-07-06 08:04:00,18576.0,90,90.0,mmHg,0.0,0.0,None,None
7886598,20563806.0,69587.0,158288.0,299998.0,223752.0,2181-07-06 20:19:00,2181-07-06 20:19:00,20053.0,90,90.0,mmHg,0.0,0.0,None,None


In [15]:
# Print some statistics about the effects of the cleaning step
df_before = chartevents_subset
df_after = chartevents_cleaning_02_sufficient_values_only

row_count_before = len(df_before)
row_count_after = len(df_after)
row_count_dif = row_count_before - row_count_after

icustay_count_before = len(df_before.ICUSTAY_ID.unique())
icustay_count_after = len(df_after.ICUSTAY_ID.unique())
icustay_count_dif = icustay_count_before - icustay_count_after

icustay_itemid_count_before = len(df_before.groupby(['ICUSTAY_ID','ITEMID']).size())
icustay_itemid_count_after = len(df_after.groupby(['ICUSTAY_ID','ITEMID']).size())
icustay_itemid_count_dif = icustay_itemid_count_before - icustay_itemid_count_after

icustay_hr_count_before = len(df_before[df_before.ITEMID.isin([220045, 220046, 220047])]['ICUSTAY_ID'].unique())
icustay_bp_count_before = len(df_before[df_before.ITEMID.isin([220179, 223751, 223752])]['ICUSTAY_ID'].unique())
icustay_o2_count_before = len(df_before[df_before.ITEMID.isin([220277, 223769, 223770])]['ICUSTAY_ID'].unique())
icustay_hr_count_after = len(df_after[df_after.ITEMID.isin([220045, 220046, 220047])]['ICUSTAY_ID'].unique())
icustay_bp_count_after = len(df_after[df_after.ITEMID.isin([220179, 223751, 223752])]['ICUSTAY_ID'].unique())
icustay_o2_count_after = len(df_after[df_after.ITEMID.isin([220277, 223769, 223770])]['ICUSTAY_ID'].unique())
icustay_hr_count_dif = icustay_hr_count_before - icustay_hr_count_after
icustay_bp_count_dif = icustay_bp_count_before - icustay_bp_count_after
icustay_o2_count_dif = icustay_o2_count_before - icustay_o2_count_after

icustay_parameter_count_before = icustay_hr_count_before + icustay_bp_count_before + icustay_o2_count_before
icustay_parameter_count_after = icustay_hr_count_after + icustay_bp_count_after + icustay_o2_count_after
icustay_parameter_count_dif = icustay_parameter_count_before - icustay_parameter_count_after

print("The length of the data frame is reduced by",f'{row_count_dif:,}',"rows from",f'{row_count_before:,}',"rows to",f'{row_count_after:,}',"rows.")
print("The number of ICUSTAY_IDs is reduced by",f'{icustay_count_dif:,}',"from",f'{icustay_count_before:,}',"to",f'{icustay_count_after:,}',"ICUSTAY_IDs.")
print("The number of ICUSTAY_ID ITEMID combinations is reduced by",f'{icustay_itemid_count_dif:,}',"from",f'{icustay_itemid_count_before:,}',"to",f'{icustay_itemid_count_after:,}',"ICUSTAY_ID ITEMID combinations.")
print("The number of ICU stay parameter combinations is reduced by",f'{icustay_parameter_count_dif:,}',"from",f'{icustay_parameter_count_before:,}',"to",f'{icustay_parameter_count_after:,}',"ICUSTAY_IDs.")
print("  HR:  ","reduced from",f'{icustay_hr_count_before:,}',"to",f'{icustay_hr_count_after:,}',"by",f'{icustay_hr_count_dif:,}')
print("  NBPs:","reduced from",f'{icustay_bp_count_before:,}',"to",f'{icustay_bp_count_after:,}',"by",f'{icustay_bp_count_dif:,}')
print("  SpO2:","reduced from",f'{icustay_o2_count_before:,}',"to",f'{icustay_o2_count_after:,}',"by",f'{icustay_o2_count_dif:,}')

The length of the data frame is reduced by 8,680 rows from 7,886,600 rows to 7,877,920 rows.
The number of ICUSTAY_IDs is reduced by 68 from 23,446 to 23,378 ICUSTAY_IDs.
The number of ICUSTAY_ID ITEMID combinations is reduced by 1,724 from 208,374 to 206,650 ICUSTAY_ID ITEMID combinations.
The number of ICU stay parameter combinations is reduced by 1,293 from 70,177 to 68,884 ICUSTAY_IDs.
  HR:   reduced from 23,445 to 23,370 by 75
  NBPs: reduced from 23,296 to 22,174 by 1,122
  SpO2: reduced from 23,436 to 23,340 by 96


In [16]:
import pandas as pd
import pyarrow as pa
path_to_dir_chartevents_clean = './reportdata/'
# Save chartevents_cleaning_02_sufficient_values_only as parquet file
chartevents_cleaning_02_sufficient_values_only.to_parquet(path_to_dir_chartevents_clean+'chartevents_cleaning_02_sufficient_values_only.parquet', engine='pyarrow')

## 3. Remove measurement values outside clinically valid ranges

Firstly, measurement values outside the clinically valid ranges are flagged in the new column `CLEANING_FLAG`.

Secondly, the affected measurement values are set to NaN, i.e. the data points are removed. In doing so, the values are not deleted from the original `VALUENUM` column. Instead, a new `VALUENUM_CLEAN` column is added to the modified CHARTEVENTS data frame, which contains the cleaned values. This column will be further used in the following cleaning steps.

### 3.1 Flag measurements outside clinically valid ranges

In [1]:
import pandas as pd
import pyarrow as pa
path_to_dir_chartevents_clean = './reportdata/'
# Read chartevents_cleaning_02_sufficient_values_only from parquet file
chartevents_cleaning_02_sufficient_values_only = pd.read_parquet(path_to_dir_chartevents_clean+'chartevents_cleaning_02_sufficient_values_only.parquet', engine='pyarrow')

In [3]:
# Clinically valid value ranges
# Heart Rate (220045): 0-350
# Non Invasive Blood Pressure systolic (220179): 0-375
# Non Invasive Blood Pressure diastolic (220180): 0-375
# O2 saturation pulseoxymetry (220277): 0-100

# Add new column CLEANING_FLAG, which is used to mark values outside the respective clinically valid range as "Below valid measurement range" or "Above valid measurement range".
import numpy as np
flagged_measurements = chartevents_cleaning_02_sufficient_values_only[['ROW_ID','ITEMID','VALUENUM']].copy()
flagged_measurements.insert(loc=len(flagged_measurements.columns), column='CLEANING_FLAG', value=np.nan)

flagged_measurements.loc[
    ((flagged_measurements['ITEMID'] == 220045) & (flagged_measurements['VALUENUM'] < 0)) | 
    ((flagged_measurements['ITEMID'] == 220179) & (flagged_measurements['VALUENUM'] < 0)) | 
    ((flagged_measurements['ITEMID'] == 220180) & (flagged_measurements['VALUENUM'] < 0)) |
    ((flagged_measurements['ITEMID'] == 220277) & (flagged_measurements['VALUENUM'] < 0)),
    'CLEANING_FLAG'] = "Below valid measurement range"

flagged_measurements.loc[
    ((flagged_measurements['ITEMID'] == 220045) & (flagged_measurements['VALUENUM'] > 350)) | 
    ((flagged_measurements['ITEMID'] == 220179) & (flagged_measurements['VALUENUM'] > 375)) | 
    ((flagged_measurements['ITEMID'] == 220180) & (flagged_measurements['VALUENUM'] > 375)) |
    ((flagged_measurements['ITEMID'] == 220277) & (flagged_measurements['VALUENUM'] > 100)),
    'CLEANING_FLAG'] = "Above valid measurement range"

# Reduce to relevant columns and rows (only those needed for merging with chartevents_cleaning_02_sufficient_values_only data frame) 
flagged_measurements = flagged_measurements.drop(['ITEMID','VALUENUM'], axis = 1).dropna().reset_index(drop=True)

display(flagged_measurements)

,ROW_ID,CLEANING_FLAG
0,23041147.0,Above valid measurement range
1,4153364.0,Above valid measurement range
2,10012784.0,Above valid measurement range
3,19249944.0,Above valid measurement range
4,19526026.0,Above valid measurement range
...,...,...
68,2621410.0,Above valid measurement range
69,14074468.0,Above valid measurement range
70,1256062.0,Above valid measurement range
71,24960577.0,Above valid measurement range


In [3]:
# Print some statistical information about the effects of the cleaning step.
measurements_affected_count = len(flagged_measurements)
measurements_affected_above_count = len(flagged_measurements[flagged_measurements.CLEANING_FLAG == "Above valid measurement range"])
measurements_affected_below_count = len(flagged_measurements[flagged_measurements.CLEANING_FLAG == "Below valid measurement range"])

print(f'{measurements_affected_count:,}',"measurements are outside the clinically valid range,",f'{measurements_affected_above_count:,}',"above and",f'{measurements_affected_below_count:,}',"below.")

73 measurements are outside the clinically valid range, 70 above and 3 below.


### 3.2 Introduce new VALUENUM_CLEAN column and set measurement values outside valid ranges to NaN

In [4]:
# Next, data frames chartevents_cleaning_02_sufficient_values_only and flagged_measurements are merged to form a new data frame chartevents_cleaning_03_measurements_in_valid_range.
# The latter will be extended/modified again in the next cleaning step and so on.

chartevents_cleaning_03_measurements_in_valid_range = chartevents_cleaning_02_sufficient_values_only.copy()
# Create new column 'VALUENUM_CLEAN' that equals 'VALUENUM' values
chartevents_cleaning_03_measurements_in_valid_range['VALUENUM_CLEAN'] = chartevents_cleaning_03_measurements_in_valid_range['VALUENUM']

# Set all 'VALUENUM_CLEAN' cells to NaN where the original values have been flagged as being below or above the valid range.
chartevents_cleaning_03_measurements_in_valid_range = chartevents_cleaning_03_measurements_in_valid_range.merge(flagged_measurements, how='left', on=['ROW_ID'])
chartevents_cleaning_03_measurements_in_valid_range.loc[
    (chartevents_cleaning_03_measurements_in_valid_range.CLEANING_FLAG == "Below valid measurement range") | 
    (chartevents_cleaning_03_measurements_in_valid_range.CLEANING_FLAG == "Above valid measurement range"),
    'VALUENUM_CLEAN'] = None

display(chartevents_cleaning_03_measurements_in_valid_range)

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,VALUENUM_CLEAN,CLEANING_FLAG
0,14005075.0,55973.0,152234.0,200001.0,220045.0,2181-11-25 19:06:00,2181-11-25 19:17:00,20622.0,115,115.0,bpm,0.0,0.0,None,None,115.0,NaN
1,14005090.0,55973.0,152234.0,200001.0,220045.0,2181-11-25 19:16:00,2181-11-25 19:16:00,20622.0,114,114.0,bpm,0.0,0.0,None,None,114.0,NaN
2,14005105.0,55973.0,152234.0,200001.0,220045.0,2181-11-25 20:00:00,2181-11-25 22:02:00,21108.0,113,113.0,bpm,0.0,0.0,None,None,113.0,NaN
3,14005111.0,55973.0,152234.0,200001.0,220045.0,2181-11-25 21:00:00,2181-11-25 22:02:00,21108.0,108,108.0,bpm,0.0,0.0,None,None,108.0,NaN
4,14005117.0,55973.0,152234.0,200001.0,220045.0,2181-11-25 22:00:00,2181-11-25 22:02:00,21108.0,110,110.0,bpm,0.0,0.0,None,None,110.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7877915,20563805.0,69587.0,158288.0,299998.0,223751.0,2181-07-06 20:19:00,2181-07-06 20:19:00,20053.0,160,160.0,mmHg,0.0,0.0,None,None,160.0,NaN
7877916,20563920.0,69587.0,158288.0,299998.0,223751.0,2181-07-07 08:00:00,2181-07-07 10:04:00,18462.0,160,160.0,mmHg,0.0,0.0,None,None,160.0,NaN
7877917,20563683.0,69587.0,158288.0,299998.0,223752.0,2181-07-06 08:02:00,2181-07-06 08:04:00,18576.0,90,90.0,mmHg,0.0,0.0,None,None,90.0,NaN
7877918,20563806.0,69587.0,158288.0,299998.0,223752.0,2181-07-06 20:19:00,2181-07-06 20:19:00,20053.0,90,90.0,mmHg,0.0,0.0,None,None,90.0,NaN


In [5]:
# For demo purposes, show rows with cleaning flag (selected columns)
chartevents_cleaning_03_measurements_in_valid_range[
    chartevents_cleaning_03_measurements_in_valid_range.CLEANING_FLAG.notnull()
    ][['ROW_ID','ICUSTAY_ID','ITEMID','CHARTTIME','VALUENUM','VALUENUM_CLEAN','CLEANING_FLAG']]

,ROW_ID,ICUSTAY_ID,ITEMID,CHARTTIME,VALUENUM,VALUENUM_CLEAN,CLEANING_FLAG
16851,23041147.0,200238.0,220277.0,2117-04-23 00:00:00,1010.00,NaN,Above valid measurement range
225482,4153364.0,202944.0,220179.0,2134-11-05 22:56:00,1148.00,NaN,Above valid measurement range
520864,10012784.0,206851.0,220277.0,2172-02-25 08:30:00,9694.00,NaN,Above valid measurement range
570048,19249944.0,207606.0,220277.0,2165-03-31 07:00:00,1005.00,NaN,Above valid measurement range
806224,19526026.0,210772.0,220045.0,2129-09-21 11:38:00,459.00,NaN,Above valid measurement range
...,...,...,...,...,...,...,...
6920798,2621410.0,289641.0,220179.0,2172-03-11 11:00:00,141146.04,NaN,Above valid measurement range
7445940,14074468.0,295569.0,220277.0,2199-12-31 00:00:00,964.00,NaN,Above valid measurement range
7690153,1256062.0,297300.0,220179.0,2155-09-24 17:00:00,1114.00,NaN,Above valid measurement range
7738012,24960577.0,298066.0,220179.0,2186-04-14 18:00:00,8582.00,NaN,Above valid measurement range


In [6]:
import pandas as pd
import pyarrow as pa
path_to_dir_chartevents_clean = './reportdata/'
# Save chartevents_cleaning_03_measurements_in_valid_range as parquet file
chartevents_cleaning_03_measurements_in_valid_range.to_parquet(path_to_dir_chartevents_clean+'chartevents_cleaning_03_measurements_in_valid_range.parquet', engine='pyarrow')

## 4. Revert local threshold swap

Short explanation of **Revert local threshold swap**:

In some cases, the thresholds appear to be exactly swapped, i.e. the threshold of type *high* appears to be actually the threshold of type *low* and vice versa. Such an apparent swap is usually not present over the entire period of the alarm threshold value series, but only at certain sections, i.e. locations. Accordingly, we call it a *local threshold swap*. Local threshold swaps are corrected by reversing the swap, i.e. by swapping the alarm threshold value of type *high* with that of type *low* at the affected locations.

More detailed explanation:

In the data of some ICU stays, the *high* and *low* alarm threshold value seris overlap at certain points. For example, during a certain period of time, the threshold for a heart rate that is too high may be below the threshold for a heart rate that is too low, and vice versa.

According to the medical experts consulted, there is no plausible reason for this. In fact, medical devices should not allow the setting of such overlapping alarm thresholds in the first place.

The overlap can show up in different ways. Roughly speaking, we have observed three variants looking at time series plots:

1. The low threshold temporarily exceeds the high threshold, while the latter continues 'normally'.
2. The high threshold value temporarily falls below the low threshold value, while the latter continues 'normally'.
3. Both thresholds temporarily overlap so that they appear swapped, which is 'abnormal' for both.

There are two sub-variants for variant (3):

- 3a The threshold values are swapped, but do not decrease/increase to the same extent, so it is not an exact swap.
- 3b The thresholds are swapped, decreasing/increasing  to the same extent, so it looks like an exact swap.

According to the agreement with medical experts, the two threshold values for case 3b (exact swap) are swapped back for the time period affected.

### 4.1 Identify candidates for local threshold swap

Coarse detection of the ICU stay/ vital parameter combinations for which the local swapping of thresholds is a possible option. Purpose of this preliminary step is to reduce the computational effort. The aim is to reduce the relatively complex threshold swap step to potentially affected cases.

The coarse detection is done by comparing the minimum *high* threshold to the maximum *low* threshold for each ICU stay/ vital parameter combination. If the minimum *high* threshold is below the maximum *low* threshold, the ICU stay/ vital parameter combination is considered for threshold swapping.

The output of this section is a data frame that includes the ICU stay/ vital parameter combinations that are candidates for swapping. It is to be expected that the number of candidates is higher than the number of swaps eventually performed. The reason for this is that not all alarm threshold overlaps represent an exact swap that can be reverted.

In [1]:
import pandas as pd
import pyarrow as pa
path_to_dir_chartevents_clean = './reportdata/'
# Read chartevents_cleaning_03_measurements_in_valid_range from parquet file
chartevents_cleaning_03_measurements_in_valid_range = pd.read_parquet(path_to_dir_chartevents_clean+'chartevents_cleaning_03_measurements_in_valid_range.parquet', engine='pyarrow')

In [2]:
import pandas as pd

parameters = pd.DataFrame({
    'LABEL':            ['HR',      'NBPs',     'SpO2'],
    'VALUE':            [220045,    220179,     220277],
    'THRESHOLD_HIGH':   [220046,    223751,     223769],
    'THRESHOLD_LOW':    [220047,    223752,     223770]})

# Create empty data frames to which will be appended during the loop
min_threshold_high_per_icustay = pd.DataFrame(columns=['ICUSTAY_ID', 'THRESHOLD_HIGH_MIN', 'ITEMID_VALUE'])
max_threshold_high_per_icustay = pd.DataFrame(columns=['ICUSTAY_ID', 'THRESHOLD_LOW_MAX', 'ITEMID_VALUE'])

for i, parameter in parameters.iterrows():

    # For current parameter, compute minimum value of high threshold for all ICU stays
    min_threshold_high = chartevents_cleaning_03_measurements_in_valid_range[
        chartevents_cleaning_03_measurements_in_valid_range['ITEMID'] == parameter['THRESHOLD_HIGH']
        ].groupby(['ICUSTAY_ID','ITEMID'])['VALUENUM'].min()
    min_threshold_high = min_threshold_high.reset_index()
    min_threshold_high = min_threshold_high[['ICUSTAY_ID','VALUENUM']].rename(columns = {'VALUENUM':'THRESHOLD_HIGH_MIN'}).assign(ITEMID_VALUE=parameter.VALUE)

    # For current parameter, compute maximium value of low threshold for all ICU stays
    max_threshold_low = chartevents_cleaning_03_measurements_in_valid_range[
        chartevents_cleaning_03_measurements_in_valid_range['ITEMID'] == parameter['THRESHOLD_LOW']
        ].groupby(['ICUSTAY_ID','ITEMID'])['VALUENUM'].max()
    max_threshold_low = max_threshold_low.reset_index()
    max_threshold_low = max_threshold_low[['ICUSTAY_ID','VALUENUM']].rename(columns = {'VALUENUM':'THRESHOLD_LOW_MAX'}).assign(ITEMID_VALUE=parameter.VALUE)

    # Append the results of the current parameter to the data frames for the overall results
    min_threshold_high_per_icustay = min_threshold_high_per_icustay.append(min_threshold_high, ignore_index=True)
    max_threshold_high_per_icustay = max_threshold_high_per_icustay.append(max_threshold_low, ignore_index=True)

# Merge data frames
threshold_min_max_per_icustay = min_threshold_high_per_icustay.merge(max_threshold_high_per_icustay, on=['ICUSTAY_ID','ITEMID_VALUE'])
threshold_min_max_per_icustay = threshold_min_max_per_icustay[['ICUSTAY_ID', 'ITEMID_VALUE', 'THRESHOLD_HIGH_MIN', 'THRESHOLD_LOW_MAX']]

display(threshold_min_max_per_icustay)

,ICUSTAY_ID,ITEMID_VALUE,THRESHOLD_HIGH_MIN,THRESHOLD_LOW_MAX
0,200001.0,220045,120.0,60.0
1,200010.0,220045,120.0,50.0
2,200011.0,220045,120.0,55.0
3,200016.0,220045,120.0,60.0
4,200021.0,220045,120.0,50.0
...,...,...,...,...
68878,299956.0,220277,100.0,90.0
68879,299957.0,220277,100.0,92.0
68880,299962.0,220277,100.0,92.0
68881,299979.0,220277,100.0,93.0


In [3]:
# Identify threshold swap candidates by comparing the minimum high threshold to the maximum low threshold for each ICU stay arameter combination.
# If the minimum low threshold is below the maximum high threshold, the ICU stay parameter combination is considered for threshold swapping. 
threshold_min_max_per_icustay['CROSS'] = threshold_min_max_per_icustay['THRESHOLD_HIGH_MIN'] < threshold_min_max_per_icustay['THRESHOLD_LOW_MAX']
threshold_swap_candidates = threshold_min_max_per_icustay[threshold_min_max_per_icustay['CROSS'] == True][['ICUSTAY_ID','ITEMID_VALUE']].reset_index(drop=True)
display(threshold_swap_candidates)

,ICUSTAY_ID,ITEMID_VALUE
0,200143.0,220045
1,200153.0,220045
2,200339.0,220045
3,200550.0,220045
4,200552.0,220045
...,...,...
1538,299387.0,220277
1539,299725.0,220277
1540,299742.0,220277
1541,299847.0,220277


### 4.2 Prepare data set for local threshold swap

In [4]:
# The threshold_swap_candidates data frame contains only the ITEMIDs of the vital sign values, not the ITEMIDs of the associated thresholds.
# To facilitate the subsequent subsetting of the CHARTEVENTS data frame, auxiliary data frames are created with the threshold ITEMIDs.
# The threshold ITEMIDs are combined into one data frame, which is then used to filter the CHARTEVENT data frame.
# There is probably a smarter way to do this, but this was the quick way.

import pandas as pd

parameters = pd.DataFrame({
    'LABEL':            ['HR',      'NBPs',     'SpO2'],
    'VALUE':            [220045,    220179,     220277],
    'THRESHOLD_HIGH':   [220046,    223751,     223769],
    'THRESHOLD_LOW':    [220047,    223752,     223770]})

# Create empty data frames to which will be appended during the loop
itemid_threshold_high_per_icustay = pd.DataFrame(columns=['ICUSTAY_ID', 'ITEMID'])
itemid_threshold_low_per_icustay = pd.DataFrame(columns=['ICUSTAY_ID', 'ITEMID'])

for i, parameter in parameters.iterrows():

    # For current parameter, create data frames with threshold ITEMIDs
    itemid_threshold_high = threshold_swap_candidates[threshold_swap_candidates['ITEMID_VALUE'] == parameter['VALUE']][['ICUSTAY_ID']].assign(ITEMID=parameter.THRESHOLD_HIGH)
    itemid_threshold_low = threshold_swap_candidates[threshold_swap_candidates['ITEMID_VALUE'] == parameter['VALUE']][['ICUSTAY_ID']].assign(ITEMID=parameter.THRESHOLD_LOW)

    # Append the results of the current parameter to the data frames for the overall results
    itemid_threshold_high_per_icustay = itemid_threshold_high_per_icustay.append(itemid_threshold_high, ignore_index=True)
    itemid_threshold_low_per_icustay = itemid_threshold_low_per_icustay.append(itemid_threshold_low, ignore_index=True)

# Merge data frames vertically
threshold_swap_filter = pd.concat([itemid_threshold_high_per_icustay, itemid_threshold_low_per_icustay], axis= 0)

# Sort to make it pretty (not important)
threshold_swap_filter = threshold_swap_filter.sort_values(by=['ICUSTAY_ID','ITEMID']).reset_index(drop=True)

display(threshold_swap_filter)

,ICUSTAY_ID,ITEMID
0,200061.0,223769
1,200061.0,223770
2,200075.0,223769
3,200075.0,223770
4,200143.0,220046
...,...,...
3081,299847.0,223770
3082,299879.0,220046
3083,299879.0,220047
3084,299889.0,223769


In [5]:
# Filter the chartevents_cleaning_03_measurements_in_valid_range based on the threshold_swap_filter
threshold_swap_data = pd.merge(chartevents_cleaning_03_measurements_in_valid_range,threshold_swap_filter)
display(threshold_swap_data)

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,VALUENUM_CLEAN,CLEANING_FLAG
0,22446631.0,74282.0,121149.0,200061.0,223769.0,2134-01-23 18:13:00,2134-01-23 18:13:00,15014.0,100,100.0,%,0.0,0.0,None,None,100.0,None
1,22446678.0,74282.0,121149.0,200061.0,223769.0,2134-01-23 20:05:00,2134-01-23 20:06:00,17609.0,100,100.0,%,0.0,0.0,None,None,100.0,None
2,22446831.0,74282.0,121149.0,200061.0,223769.0,2134-01-24 08:13:00,2134-01-24 09:13:00,15014.0,100,100.0,%,0.0,0.0,None,None,100.0,None
3,22446923.0,74282.0,121149.0,200061.0,223769.0,2134-01-24 20:34:00,2134-01-24 20:35:00,20117.0,35,35.0,%,0.0,0.0,None,None,35.0,None
4,22447074.0,74282.0,121149.0,200061.0,223769.0,2134-01-25 08:44:00,2134-01-25 08:44:00,14518.0,100,100.0,%,0.0,0.0,None,None,100.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69997,12445279.0,54358.0,134569.0,299889.0,223770.0,2172-06-06 08:00:00,2172-06-06 07:21:00,18959.0,92,92.0,%,0.0,0.0,None,None,92.0,None
69998,12445425.0,54358.0,134569.0,299889.0,223770.0,2172-06-06 20:00:00,2172-06-06 20:27:00,14576.0,92,92.0,%,0.0,0.0,None,None,92.0,None
69999,12445599.0,54358.0,134569.0,299889.0,223770.0,2172-06-07 08:00:00,2172-06-07 07:45:00,16139.0,92,92.0,%,0.0,0.0,None,None,92.0,None
70000,12445763.0,54358.0,134569.0,299889.0,223770.0,2172-06-07 20:00:00,2172-06-07 19:43:00,14576.0,92,92.0,%,0.0,0.0,None,None,92.0,None


### 4.3 Perform local threshold swap

In [7]:
import numpy as np
import pandas as pd

parameters = pd.DataFrame({
    'LABEL':            ['HR',      'NBPs',     'SpO2'],
    'VALUE':            [220045,    220179,     220277],
    'THRESHOLD_HIGH':   [220046,    223751,     223769],
    'THRESHOLD_LOW':    [220047,    223752,     223770]})

icustays = threshold_swap_data.ICUSTAY_ID.unique()

thresholds_fixed = pd.DataFrame(columns=['ICUSTAY_ID', 'ITEMID_VALUE', 'CHARTTIME', 'THRESHOLD_LOW_FIXED', 'THRESHOLD_HIGH_FIXED'])

for icustay in icustays:
    
    for i, parameter in parameters.iterrows():
        
        threshold_high = threshold_swap_data[
            (threshold_swap_data["ICUSTAY_ID"] == icustay) & 
            (threshold_swap_data["ITEMID"] == parameter['THRESHOLD_HIGH'])][
            ['CHARTTIME','VALUENUM']
            ].sort_values(by=['CHARTTIME']).rename(columns = {'VALUENUM':'THRESHOLD_HIGH'})

        threshold_low = threshold_swap_data[
            (threshold_swap_data["ICUSTAY_ID"] == icustay) & 
            (threshold_swap_data["ITEMID"] == parameter['THRESHOLD_LOW'])][
            ['CHARTTIME','VALUENUM']
            ].sort_values(by=['CHARTTIME']).rename(columns = {'VALUENUM':'THRESHOLD_LOW'})

        thresholds_by_icustay_parameter_charttime = threshold_high.merge(threshold_low, on=['CHARTTIME']).assign(ICUSTAY_ID=icustay, ITEMID_VALUE=parameter.VALUE)
        
        # Create a new column that contains the chronologically following threshold value of the same type
        thresholds_by_icustay_parameter_charttime['THRESHOLD_HIGH_NEXT'] = thresholds_by_icustay_parameter_charttime['THRESHOLD_HIGH'].shift(-1)
        thresholds_by_icustay_parameter_charttime['THRESHOLD_LOW_NEXT'] = thresholds_by_icustay_parameter_charttime['THRESHOLD_LOW'].shift(-1)

        thresholds_by_icustay_parameter_charttime['DIF_THRESHOLD_HIGH_NEXT'] = thresholds_by_icustay_parameter_charttime['THRESHOLD_HIGH_NEXT'] - thresholds_by_icustay_parameter_charttime['THRESHOLD_HIGH']
        thresholds_by_icustay_parameter_charttime['DIF_THRESHOLD_LOW_NEXT'] = thresholds_by_icustay_parameter_charttime['THRESHOLD_LOW_NEXT'] - thresholds_by_icustay_parameter_charttime['THRESHOLD_LOW']
        

        thresholds_by_icustay_parameter_charttime.insert(loc=len(thresholds_by_icustay_parameter_charttime.columns), column='THRESHOLD_HIGH_FIXED', value=np.nan)
        thresholds_by_icustay_parameter_charttime.insert(loc=len(thresholds_by_icustay_parameter_charttime.columns), column='THRESHOLD_LOW_FIXED', value=np.nan)

        thresholds_by_icustay_parameter_charttime.loc[
            (thresholds_by_icustay_parameter_charttime['THRESHOLD_HIGH'] < thresholds_by_icustay_parameter_charttime['THRESHOLD_LOW']) &
            (abs(thresholds_by_icustay_parameter_charttime['DIF_THRESHOLD_HIGH_NEXT']) == abs(thresholds_by_icustay_parameter_charttime['DIF_THRESHOLD_LOW_NEXT'])),
            'THRESHOLD_HIGH_FIXED'] = thresholds_by_icustay_parameter_charttime['THRESHOLD_LOW']

        thresholds_by_icustay_parameter_charttime.loc[
            (thresholds_by_icustay_parameter_charttime['THRESHOLD_HIGH'] < thresholds_by_icustay_parameter_charttime['THRESHOLD_LOW']) &
            (abs(thresholds_by_icustay_parameter_charttime['DIF_THRESHOLD_HIGH_NEXT']) == abs(thresholds_by_icustay_parameter_charttime['DIF_THRESHOLD_LOW_NEXT'])),
            'THRESHOLD_LOW_FIXED'] = thresholds_by_icustay_parameter_charttime['THRESHOLD_HIGH']
        
        thresholds_by_icustay_parameter_charttime.dropna(inplace=True)
        thresholds_fixed_for_icustayid_itemid = thresholds_by_icustay_parameter_charttime[['ICUSTAY_ID', 'ITEMID_VALUE', 'CHARTTIME', 'THRESHOLD_LOW_FIXED', 'THRESHOLD_HIGH_FIXED']]

        thresholds_fixed = thresholds_fixed.append(thresholds_fixed_for_icustayid_itemid, ignore_index=True)

display(thresholds_fixed)

,ICUSTAY_ID,ITEMID_VALUE,CHARTTIME,THRESHOLD_LOW_FIXED,THRESHOLD_HIGH_FIXED
0,200550.0,220277,2182-08-03 20:00:00,100.0,921.0
1,200552.0,220045,2149-10-06 08:00:00,60.0,150.0
2,200552.0,220045,2149-10-06 16:00:00,60.0,150.0
3,201220.0,220277,2106-04-29 08:00:00,90.0,100.0
4,201360.0,220179,2149-01-17 19:52:00,90.0,160.0
...,...,...,...,...,...
430,297549.0,220045,2165-12-27 12:00:00,60.0,120.0
431,297714.0,220045,2198-10-04 08:00:00,50.0,130.0
432,297767.0,220045,2165-04-26 08:00:00,50.0,120.0
433,297767.0,220045,2165-04-26 16:00:00,50.0,120.0


In [8]:
# In the next step, data frames chartevents_cleaning_03_measurements_in_valid_range and thresholds_fixed are merged ...
# ... to form a new data frame chartevents_cleaning_04_exact_threshold_swaps_reverted.
# The latter will be extended/modified again in the next cleaning step and so on.
chartevents_cleaning_04_exact_threshold_swaps_reverted = chartevents_cleaning_03_measurements_in_valid_range.copy()

In [10]:
# Unnecessarily complicated step; could be simplified by better preparation of the data frame to be merged.
# Needed because the thresholds_fixed data frame does contain the ITEMID_VALUE (i.e. the ITEMIDs of the vital parameters) but not the alarm threshold ITEMIDs.
import pandas as pd
import numpy as np

parameters = pd.DataFrame({
    'LABEL':            ['HR',      'NBPs',     'SpO2'],
    'VALUE':            [220045,    220179,     220277],
    'THRESHOLD_HIGH':   [220046,    223751,     223769],
    'THRESHOLD_LOW':    [220047,    223752,     223770]})

thresholds_fixed.insert(loc=len(thresholds_fixed.columns), column='ITEMID_THRESHOLD_HIGH', value=np.nan)
thresholds_fixed.insert(loc=len(thresholds_fixed.columns), column='ITEMID_THRESHOLD_LOW', value=np.nan)

for i, parameter in parameters.iterrows():

    thresholds_fixed.loc[thresholds_fixed.ITEMID_VALUE == parameter['VALUE'], 'ITEMID_THRESHOLD_HIGH'] = parameter['THRESHOLD_HIGH']
    thresholds_fixed.loc[thresholds_fixed.ITEMID_VALUE == parameter['VALUE'], 'ITEMID_THRESHOLD_LOW'] = parameter['THRESHOLD_LOW']

display(thresholds_fixed)

,ICUSTAY_ID,ITEMID_VALUE,CHARTTIME,THRESHOLD_LOW_FIXED,THRESHOLD_HIGH_FIXED,ITEMID_THRESHOLD_HIGH,ITEMID_THRESHOLD_LOW
0,200550.0,220277,2182-08-03 20:00:00,100.0,921.0,223769.0,223770.0
1,200552.0,220045,2149-10-06 08:00:00,60.0,150.0,220046.0,220047.0
2,200552.0,220045,2149-10-06 16:00:00,60.0,150.0,220046.0,220047.0
3,201220.0,220277,2106-04-29 08:00:00,90.0,100.0,223769.0,223770.0
4,201360.0,220179,2149-01-17 19:52:00,90.0,160.0,223751.0,223752.0
...,...,...,...,...,...,...,...
430,297549.0,220045,2165-12-27 12:00:00,60.0,120.0,220046.0,220047.0
431,297714.0,220045,2198-10-04 08:00:00,50.0,130.0,220046.0,220047.0
432,297767.0,220045,2165-04-26 08:00:00,50.0,120.0,220046.0,220047.0
433,297767.0,220045,2165-04-26 16:00:00,50.0,120.0,220046.0,220047.0


In [11]:
thresholds_fixed_high = thresholds_fixed[
    ['ICUSTAY_ID','CHARTTIME','THRESHOLD_HIGH_FIXED','ITEMID_THRESHOLD_HIGH']
    ].rename(columns = {'THRESHOLD_HIGH_FIXED':'VALUENUM_CLEAN', 'ITEMID_THRESHOLD_HIGH':'ITEMID'})
thresholds_fixed_high = thresholds_fixed_high[['ICUSTAY_ID','ITEMID','CHARTTIME','VALUENUM_CLEAN']]

thresholds_fixed_low = thresholds_fixed[
    ['ICUSTAY_ID','CHARTTIME','THRESHOLD_LOW_FIXED','ITEMID_THRESHOLD_LOW']
    ].rename(columns = {'THRESHOLD_LOW_FIXED':'VALUENUM_CLEAN', 'ITEMID_THRESHOLD_LOW':'ITEMID'})
thresholds_fixed_low = thresholds_fixed_low[['ICUSTAY_ID','ITEMID','CHARTTIME','VALUENUM_CLEAN']]

In [ ]:
# Takes quite long; there is probably a better why to insert/replace the swapped threshold values in the VALUENUM_CLEAN column

for i, row in thresholds_fixed_high.iterrows():

    chartevents_cleaning_04_exact_threshold_swaps_reverted.loc[
        (chartevents_cleaning_04_exact_threshold_swaps_reverted.ICUSTAY_ID == row['ICUSTAY_ID']) &
        (chartevents_cleaning_04_exact_threshold_swaps_reverted.ITEMID == row['ITEMID']) &
        (chartevents_cleaning_04_exact_threshold_swaps_reverted.CHARTTIME == row['CHARTTIME']), 
        'VALUENUM_CLEAN'] = row['VALUENUM_CLEAN']

    chartevents_cleaning_04_exact_threshold_swaps_reverted.loc[
        (chartevents_cleaning_04_exact_threshold_swaps_reverted.ICUSTAY_ID == row['ICUSTAY_ID']) &
        (chartevents_cleaning_04_exact_threshold_swaps_reverted.ITEMID == row['ITEMID']) &
        (chartevents_cleaning_04_exact_threshold_swaps_reverted.CHARTTIME == row['CHARTTIME']), 
        'CLEANING_FLAG'] = "High threshold fixed by swap"

for i, row in thresholds_fixed_low.iterrows():

    chartevents_cleaning_04_exact_threshold_swaps_reverted.loc[
        (chartevents_cleaning_04_exact_threshold_swaps_reverted.ICUSTAY_ID == row['ICUSTAY_ID']) &
        (chartevents_cleaning_04_exact_threshold_swaps_reverted.ITEMID == row['ITEMID']) &
        (chartevents_cleaning_04_exact_threshold_swaps_reverted.CHARTTIME == row['CHARTTIME']), 
        'VALUENUM_CLEAN'] = row['VALUENUM_CLEAN']

    chartevents_cleaning_04_exact_threshold_swaps_reverted.loc[
        (chartevents_cleaning_04_exact_threshold_swaps_reverted.ICUSTAY_ID == row['ICUSTAY_ID']) &
        (chartevents_cleaning_04_exact_threshold_swaps_reverted.ITEMID == row['ITEMID']) &
        (chartevents_cleaning_04_exact_threshold_swaps_reverted.CHARTTIME == row['CHARTTIME']), 
        'CLEANING_FLAG'] = "Low threshold fixed by swap"

display(chartevents_cleaning_04_exact_threshold_swaps_reverted[
    chartevents_cleaning_04_exact_threshold_swaps_reverted.CLEANING_FLAG.isin(['High threshold fixed by swap','Low threshold fixed by swap'])])

In [19]:
import pandas as pd
import pyarrow as pa
path_to_dir_chartevents_clean = './reportdata/'
# Save chartevents_cleaning_04_exact_threshold_swaps_reverted as parquet file
chartevents_cleaning_04_exact_threshold_swaps_reverted.to_parquet(path_to_dir_chartevents_clean+'chartevents_cleaning_04_exact_threshold_swaps_reverted.parquet', engine='pyarrow')

## 5. Remove alarm threshold values outside clinically valid ranges

* If the alarm threshold value is out of the respective clinically valid range set it to NaN, i.e. remove the data point, which reduces the sampling rate of the affected alarm threshold value series.
* Currently, the valid alarm threshold value ranges are identical to the corresponding vital parameter value ranges, and they are the same for both threshold types (low and high).

In [3]:
import pandas as pd
import pyarrow as pa
path_to_dir_chartevents_clean = './reportdata/'
# Read chartevents_cleaning_04_exact_threshold_swaps_reverted from parquet file to pandas data frame
chartevents_cleaning_04_exact_threshold_swaps_reverted = pd.read_parquet(path_to_dir_chartevents_clean+'chartevents_cleaning_04_exact_threshold_swaps_reverted.parquet', engine='pyarrow')

In [5]:
# Clinically valid threshold ranges
# Currently, identical to the valid vital parameter value ranges and the same for both threshold types (low and high).
# It is conceivable to define the ranges for each threshold separately, although we currently lack the medical knowledge base for this.
# Heart rate Alarm - High (220046): 0-350
# Heart Rate Alarm - Low (220047): 0-350
# Non-Invasive Blood Pressure Alarm - High (223751): 0-375
# Non-Invasive Blood Pressure Alarm - Low (223752): 0-375
# O2 Saturation Pulseoxymetry Alarm - High (223769): 0-100
# O2 Saturation Pulseoxymetry Alarm - Low (223770): 0-100

# The approach of alarm threshold value flagging is similar to the vital parameter value flagging performed above.
# However, in this case not the original VALUENUM is used but the VALUENUM_CLEAN.
# The reason for this is that alarm threshold values outside the valid ranges can also be among the swapped back alarm threshold values.
# This is done to include the reverted exact threshold swaps (see above), which may contain thresholds outside the valid ranges.

chartevents_cleaning_05_thresholds_in_valid_range = chartevents_cleaning_04_exact_threshold_swaps_reverted.copy()

# Multiple cleaning rules may apply to a row. There are thresholds that are first swapped and then removed because they are outside the valid range.
# Both steps are tracked in the CLEANING_FLAG column. Existing CLEANING_FLAG values such as 'High threshold fixed by swap' are not overwritten but extended, e.g.'High threshold fixed by swap; Above valid threshold range'.

chartevents_cleaning_05_thresholds_in_valid_range.loc[
    (
        ((chartevents_cleaning_05_thresholds_in_valid_range['ITEMID'].isin([220046, 220047])) & (chartevents_cleaning_05_thresholds_in_valid_range['VALUENUM_CLEAN'] < 0)) |
        ((chartevents_cleaning_05_thresholds_in_valid_range['ITEMID'].isin([223751, 223752])) & (chartevents_cleaning_05_thresholds_in_valid_range['VALUENUM_CLEAN'] < 0)) |
        ((chartevents_cleaning_05_thresholds_in_valid_range['ITEMID'].isin([223769, 223770])) & (chartevents_cleaning_05_thresholds_in_valid_range['VALUENUM_CLEAN'] < 0))
    ) & (chartevents_cleaning_05_thresholds_in_valid_range['CLEANING_FLAG'].isin(['High threshold fixed by swap', 'Low threshold fixed by swap'])),
    'CLEANING_FLAG'] = chartevents_cleaning_05_thresholds_in_valid_range['CLEANING_FLAG'].astype(str)+"; Below valid threshold range"

chartevents_cleaning_05_thresholds_in_valid_range.loc[
    (
        ((chartevents_cleaning_05_thresholds_in_valid_range['ITEMID'].isin([220046, 220047])) & (chartevents_cleaning_05_thresholds_in_valid_range['VALUENUM_CLEAN'] < 0)) |
        ((chartevents_cleaning_05_thresholds_in_valid_range['ITEMID'].isin([223751, 223752])) & (chartevents_cleaning_05_thresholds_in_valid_range['VALUENUM_CLEAN'] < 0)) |
        ((chartevents_cleaning_05_thresholds_in_valid_range['ITEMID'].isin([223769, 223770])) & (chartevents_cleaning_05_thresholds_in_valid_range['VALUENUM_CLEAN'] < 0))
    ) & (chartevents_cleaning_05_thresholds_in_valid_range['CLEANING_FLAG'].isnull()),
    'CLEANING_FLAG'] = "Below valid threshold range"

chartevents_cleaning_05_thresholds_in_valid_range.loc[
    (
        ((chartevents_cleaning_05_thresholds_in_valid_range['ITEMID'].isin([220046, 220047])) & (chartevents_cleaning_05_thresholds_in_valid_range['VALUENUM_CLEAN'] > 350)) |
        ((chartevents_cleaning_05_thresholds_in_valid_range['ITEMID'].isin([223751, 223752])) & (chartevents_cleaning_05_thresholds_in_valid_range['VALUENUM_CLEAN'] > 375)) |
        ((chartevents_cleaning_05_thresholds_in_valid_range['ITEMID'].isin([223769, 223770])) & (chartevents_cleaning_05_thresholds_in_valid_range['VALUENUM_CLEAN'] > 100))
    ) & (chartevents_cleaning_05_thresholds_in_valid_range['CLEANING_FLAG'].isin(['High threshold fixed by swap', 'Low threshold fixed by swap'])),
    'CLEANING_FLAG'] = chartevents_cleaning_05_thresholds_in_valid_range['CLEANING_FLAG'].astype(str)+"; Above valid threshold range"

chartevents_cleaning_05_thresholds_in_valid_range.loc[
    (
        ((chartevents_cleaning_05_thresholds_in_valid_range['ITEMID'].isin([220046, 220047])) & (chartevents_cleaning_05_thresholds_in_valid_range['VALUENUM_CLEAN'] > 350)) |
        ((chartevents_cleaning_05_thresholds_in_valid_range['ITEMID'].isin([223751, 223752])) & (chartevents_cleaning_05_thresholds_in_valid_range['VALUENUM_CLEAN'] > 375)) |
        ((chartevents_cleaning_05_thresholds_in_valid_range['ITEMID'].isin([223769, 223770])) & (chartevents_cleaning_05_thresholds_in_valid_range['VALUENUM_CLEAN'] > 100))
    ) & (chartevents_cleaning_05_thresholds_in_valid_range['CLEANING_FLAG'].isnull()),
    'CLEANING_FLAG'] = "Above valid threshold range"

chartevents_cleaning_05_thresholds_in_valid_range.loc[
    (chartevents_cleaning_05_thresholds_in_valid_range.CLEANING_FLAG.str.contains('Below valid threshold range')) |
    (chartevents_cleaning_05_thresholds_in_valid_range.CLEANING_FLAG.str.contains('Above valid threshold range')),
    'VALUENUM_CLEAN'] = None

display(chartevents_cleaning_05_thresholds_in_valid_range[
    (chartevents_cleaning_05_thresholds_in_valid_range.CLEANING_FLAG.str.contains('Below valid threshold range')) |
    (chartevents_cleaning_05_thresholds_in_valid_range.CLEANING_FLAG.str.contains('Above valid threshold range'))
    ])

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,VALUENUM_CLEAN,CLEANING_FLAG
24044,11355583.0,51839.0,150324.0,200339.0,220047.0,2192-02-23 08:13:00,2192-02-23 08:14:00,14597.0,500,500.0,bpm,0.0,0.0,None,None,NaN,Above valid threshold range
36338,19557273.0,67348.0,179548.0,200550.0,223769.0,2182-08-03 20:00:00,2182-08-03 19:48:00,18305.0,100,100.0,%,0.0,0.0,None,None,NaN,High threshold fixed by swap; Above valid thre...
36371,19557370.0,67348.0,179548.0,200550.0,223770.0,2182-08-04 05:14:00,2182-08-04 05:14:00,18305.0,921,921.0,%,0.0,0.0,None,None,NaN,Above valid threshold range
51670,20400719.0,69237.0,138300.0,200571.0,223769.0,2150-12-23 08:00:00,2150-12-23 08:44:00,17457.0,199,199.0,%,0.0,0.0,None,None,NaN,Above valid threshold range
64276,30900042.0,91563.0,155738.0,200737.0,223769.0,2179-08-18 08:00:00,2179-08-18 08:02:00,15083.0,101,101.0,%,0.0,0.0,None,None,NaN,Above valid threshold range
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7818959,22210210.0,73579.0,159680.0,299144.0,223769.0,2153-10-22 08:15:00,2153-10-22 08:16:00,14772.0,120,120.0,%,0.0,0.0,None,None,NaN,Above valid threshold range
7827184,27456991.0,84534.0,151051.0,299305.0,223751.0,2128-01-05 20:00:00,2128-01-05 20:37:00,18901.0,1600,1600.0,mmHg,0.0,0.0,None,None,NaN,Above valid threshold range
7836742,9687038.0,48451.0,177942.0,299457.0,223769.0,2186-10-07 08:25:00,2186-10-07 08:25:00,17582.0,160,160.0,%,0.0,0.0,None,None,NaN,Above valid threshold range
7838628,4791244.0,31381.0,117080.0,299482.0,223769.0,2101-09-16 08:00:00,2101-09-16 07:56:00,14357.0,100185,100185.0,%,0.0,0.0,None,None,NaN,Above valid threshold range


In [7]:
import pandas as pd
import pyarrow as pa
path_to_dir_chartevents_clean = './reportdata/'
# Save chartevents_cleaning_05_thresholds_in_valid_range as parquet file
chartevents_cleaning_05_thresholds_in_valid_range.to_parquet(path_to_dir_chartevents_clean+'chartevents_cleaning_05_thresholds_in_valid_range.parquet', engine='pyarrow')

## 6. Remove threshold values due to local overlap

Besides the exact alarm threshold swaps addressed above, there are also cases where the alarm threshold value series of type *high* and *low* overlap, but are not exactly swapped (e.g. the alarm threshold value of the type *high* is set unreasonably low and falls below the alarm threshold value of type *low*, while the latter seems to have a correct value). Such an overlap is usually not present over the entire period of the alarm threshold value series, but only at certain sections, i.e. locations. Accordingly, we call it a *local overlap*. These cases cannot be corrected by swapping. Also, it is not possible (without a lot of effort) to determine whether only one of the two thresholds is unreasonably high/low or both. Therefore, the threshold values of both types (high and low) are removed at those locations where they overlap. Hence, we call this step *Remove threshold values due to local overlap*.

### 6.1 Identify candidates for threshold removal due to local overlap

The approach to identifying candidates for threshold removal due to local overlap is similar to the candidate identification performed above in the context of threshold swapping. However, in this case not the original VALUENUM is used but the VALUENUM_CLEAN.

In [1]:
import pandas as pd
import pyarrow as pa
path_to_dir_chartevents_clean = './reportdata/'
# Read chartevents_cleaning_05_thresholds_in_valid_range from parquet file
chartevents_cleaning_05_thresholds_in_valid_range = pd.read_parquet(path_to_dir_chartevents_clean+'chartevents_cleaning_05_thresholds_in_valid_range.parquet', engine='pyarrow')

In [2]:
import pandas as pd

parameters = pd.DataFrame({
    'LABEL':            ['HR',      'NBPs',     'SpO2'],
    'VALUE':            [220045,    220179,     220277],
    'THRESHOLD_HIGH':   [220046,    223751,     223769],
    'THRESHOLD_LOW':    [220047,    223752,     223770]})

# Create empty data frames which will be appended during the loop
min_threshold_high_per_icustay = pd.DataFrame(columns=['ICUSTAY_ID', 'THRESHOLD_HIGH_MIN', 'ITEMID_VALUE'])
max_threshold_high_per_icustay = pd.DataFrame(columns=['ICUSTAY_ID', 'THRESHOLD_LOW_MAX', 'ITEMID_VALUE'])

for i, parameter in parameters.iterrows():

    # For current parameter, compute minimum value of high threshold for all ICU stays
    min_threshold_high = chartevents_cleaning_05_thresholds_in_valid_range[
        chartevents_cleaning_05_thresholds_in_valid_range['ITEMID'] == parameter['THRESHOLD_HIGH']
        ].groupby(['ICUSTAY_ID','ITEMID'])['VALUENUM_CLEAN'].min()
    min_threshold_high = min_threshold_high.reset_index()
    min_threshold_high = min_threshold_high[['ICUSTAY_ID','VALUENUM_CLEAN']].rename(columns = {'VALUENUM_CLEAN':'THRESHOLD_HIGH_MIN'}).assign(ITEMID_VALUE=parameter.VALUE)

    # For current parameter, compute maximium value of low threshold for all ICU stays
    max_threshold_low = chartevents_cleaning_05_thresholds_in_valid_range[
        chartevents_cleaning_05_thresholds_in_valid_range['ITEMID'] == parameter['THRESHOLD_LOW']
        ].groupby(['ICUSTAY_ID','ITEMID'])['VALUENUM_CLEAN'].max()
    max_threshold_low = max_threshold_low.reset_index()
    max_threshold_low = max_threshold_low[['ICUSTAY_ID','VALUENUM_CLEAN']].rename(columns = {'VALUENUM_CLEAN':'THRESHOLD_LOW_MAX'}).assign(ITEMID_VALUE=parameter.VALUE)

    # Append the results of the current parameter to the data frames for the overall results
    min_threshold_high_per_icustay = min_threshold_high_per_icustay.append(min_threshold_high, ignore_index=True)
    max_threshold_high_per_icustay = max_threshold_high_per_icustay.append(max_threshold_low, ignore_index=True)

# Merge data frames
threshold_min_max_per_icustay = min_threshold_high_per_icustay.merge(max_threshold_high_per_icustay, on=['ICUSTAY_ID','ITEMID_VALUE'])
threshold_min_max_per_icustay = threshold_min_max_per_icustay[['ICUSTAY_ID', 'ITEMID_VALUE', 'THRESHOLD_HIGH_MIN', 'THRESHOLD_LOW_MAX']]

display(threshold_min_max_per_icustay)

,ICUSTAY_ID,ITEMID_VALUE,THRESHOLD_HIGH_MIN,THRESHOLD_LOW_MAX
0,200001.0,220045,120.0,60.0
1,200010.0,220045,120.0,50.0
2,200011.0,220045,120.0,55.0
3,200016.0,220045,120.0,60.0
4,200021.0,220045,120.0,50.0
...,...,...,...,...
68878,299956.0,220277,100.0,90.0
68879,299957.0,220277,100.0,92.0
68880,299962.0,220277,100.0,92.0
68881,299979.0,220277,100.0,93.0


In [3]:
# Identify threshold removal candidates by comparing the minimum high threshold to the maximum low threshold for each ICU stay vital parameter combination.
# If the minimum low threshold is below the maximum high threshold, the ICU stay vital parameter combination is considered for threshold removal. 
threshold_min_max_per_icustay['CROSS'] = threshold_min_max_per_icustay['THRESHOLD_HIGH_MIN'] < threshold_min_max_per_icustay['THRESHOLD_LOW_MAX']
threshold_removal_candidates = threshold_min_max_per_icustay[threshold_min_max_per_icustay['CROSS'] == True][['ICUSTAY_ID','ITEMID_VALUE']].reset_index(drop=True)
display(threshold_removal_candidates)

,ICUSTAY_ID,ITEMID_VALUE
0,200143.0,220045
1,200153.0,220045
2,200550.0,220045
3,200571.0,220045
4,200696.0,220045
...,...,...
1101,299387.0,220277
1102,299725.0,220277
1103,299742.0,220277
1104,299847.0,220277


### 6.2 Prepare data set for threshold removal due to local overlap

In [4]:
# The threshold_removal_candidates data frame contains only the ITEMIDs of the vital parameter values, not the ITEMIDs of the associated thresholds.
# To facilitate the subsequent subsetting, auxiliary data frames are created with the threshold ITEMIDs.
# The threshold ITEMIDs are combined into one data frame, which is then used to filter the data frame.
# There is probably a smarter way to do this, but this was fast enough.

import pandas as pd

parameters = pd.DataFrame({
    'LABEL':            ['HR',      'NBPs',     'SpO2'],
    'VALUE':            [220045,    220179,     220277],
    'THRESHOLD_HIGH':   [220046,    223751,     223769],
    'THRESHOLD_LOW':    [220047,    223752,     223770]})

# Create empty data frames to which will be appended during the loop
itemid_threshold_high_per_icustay = pd.DataFrame(columns=['ICUSTAY_ID', 'ITEMID'])
itemid_threshold_low_per_icustay = pd.DataFrame(columns=['ICUSTAY_ID', 'ITEMID'])

for i, parameter in parameters.iterrows():

    # For current parameter, create data frames with threshold ITEMIDs
    itemid_threshold_high = threshold_removal_candidates[threshold_removal_candidates['ITEMID_VALUE'] == parameter['VALUE']][['ICUSTAY_ID']].assign(ITEMID=parameter.THRESHOLD_HIGH)
    itemid_threshold_low = threshold_removal_candidates[threshold_removal_candidates['ITEMID_VALUE'] == parameter['VALUE']][['ICUSTAY_ID']].assign(ITEMID=parameter.THRESHOLD_LOW)

    # Append the results of the current parameter to the data frames for the overall results
    itemid_threshold_high_per_icustay = itemid_threshold_high_per_icustay.append(itemid_threshold_high, ignore_index=True)
    itemid_threshold_low_per_icustay = itemid_threshold_low_per_icustay.append(itemid_threshold_low, ignore_index=True)

# Merge data frames vertically
threshold_removal_filter = pd.concat([itemid_threshold_high_per_icustay, itemid_threshold_low_per_icustay], axis= 0)

# Sort to make it pretty (not important)
threshold_removal_filter = threshold_removal_filter.sort_values(by=['ICUSTAY_ID','ITEMID']).reset_index(drop=True)

display(threshold_removal_filter)

,ICUSTAY_ID,ITEMID
0,200061.0,223769
1,200061.0,223770
2,200075.0,223769
3,200075.0,223770
4,200143.0,220046
...,...,...
2207,299742.0,223770
2208,299847.0,223769
2209,299847.0,223770
2210,299889.0,223769


In [5]:
# Filter the chartevents_cleaning_05_thresholds_in_valid_range based on the threshold_removal_filter
threshold_removal_data = pd.merge(chartevents_cleaning_05_thresholds_in_valid_range,threshold_removal_filter)
display(threshold_removal_data)

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,VALUENUM_CLEAN,CLEANING_FLAG
0,22446631.0,74282.0,121149.0,200061.0,223769.0,2134-01-23 18:13:00,2134-01-23 18:13:00,15014.0,100,100.0,%,0.0,0.0,None,None,100.0,None
1,22446678.0,74282.0,121149.0,200061.0,223769.0,2134-01-23 20:05:00,2134-01-23 20:06:00,17609.0,100,100.0,%,0.0,0.0,None,None,100.0,None
2,22446831.0,74282.0,121149.0,200061.0,223769.0,2134-01-24 08:13:00,2134-01-24 09:13:00,15014.0,100,100.0,%,0.0,0.0,None,None,100.0,None
3,22446923.0,74282.0,121149.0,200061.0,223769.0,2134-01-24 20:34:00,2134-01-24 20:35:00,20117.0,35,35.0,%,0.0,0.0,None,None,35.0,None
4,22447074.0,74282.0,121149.0,200061.0,223769.0,2134-01-25 08:44:00,2134-01-25 08:44:00,14518.0,100,100.0,%,0.0,0.0,None,None,100.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48593,12445279.0,54358.0,134569.0,299889.0,223770.0,2172-06-06 08:00:00,2172-06-06 07:21:00,18959.0,92,92.0,%,0.0,0.0,None,None,92.0,None
48594,12445425.0,54358.0,134569.0,299889.0,223770.0,2172-06-06 20:00:00,2172-06-06 20:27:00,14576.0,92,92.0,%,0.0,0.0,None,None,92.0,None
48595,12445599.0,54358.0,134569.0,299889.0,223770.0,2172-06-07 08:00:00,2172-06-07 07:45:00,16139.0,92,92.0,%,0.0,0.0,None,None,92.0,None
48596,12445763.0,54358.0,134569.0,299889.0,223770.0,2172-06-07 20:00:00,2172-06-07 19:43:00,14576.0,92,92.0,%,0.0,0.0,None,None,92.0,None


### 6.3 Perform threshold removal due to local overlap

* Where alarm threshold values overlap, set both threshold values (high and low) to NaN for the affected time period, which reduces sampling rate of both alarm threshold value series (high and low).
* This cleaning step may lead to incorrect assumptions due to later interpolation of alarm threshold values series.

In [6]:
import numpy as np
import pandas as pd

parameters = pd.DataFrame({
    'LABEL':            ['HR',      'NBPs',     'SpO2'],
    'VALUE':            [220045,    220179,     220277],
    'THRESHOLD_HIGH':   [220046,    223751,     223769],
    'THRESHOLD_LOW':    [220047,    223752,     223770]})

icustays = threshold_removal_data.ICUSTAY_ID.unique()

thresholds_to_be_removed = pd.DataFrame(columns=['ICUSTAY_ID', 'ITEMID_VALUE', 'CHARTTIME'])

for icustay in icustays:
    
    for i, parameter in parameters.iterrows():
        
        threshold_high = threshold_removal_data[
            (threshold_removal_data["ICUSTAY_ID"] == icustay) & 
            (threshold_removal_data["ITEMID"] == parameter['THRESHOLD_HIGH'])][
            ['CHARTTIME','VALUENUM_CLEAN']
            ].sort_values(by=['CHARTTIME']).rename(columns = {'VALUENUM_CLEAN':'THRESHOLD_HIGH'})

        threshold_low = threshold_removal_data[
            (threshold_removal_data["ICUSTAY_ID"] == icustay) & 
            (threshold_removal_data["ITEMID"] == parameter['THRESHOLD_LOW'])][
            ['CHARTTIME','VALUENUM_CLEAN']
            ].sort_values(by=['CHARTTIME']).rename(columns = {'VALUENUM_CLEAN':'THRESHOLD_LOW'})

        thresholds_by_icustay_parameter_charttime = threshold_high.merge(threshold_low, on=['CHARTTIME']).assign(ICUSTAY_ID=icustay, ITEMID_VALUE=parameter.VALUE)

        thresholds_by_icustay_parameter_charttime.insert(loc=len(thresholds_by_icustay_parameter_charttime.columns), column='CLEANING_FLAG', value=np.nan)
        thresholds_by_icustay_parameter_charttime.loc[
            (thresholds_by_icustay_parameter_charttime['THRESHOLD_HIGH'] < thresholds_by_icustay_parameter_charttime['THRESHOLD_LOW']),
            'CLEANING_FLAG'] = "Threshold removal due to overlap"
        
        thresholds_by_icustay_parameter_charttime.dropna(inplace=True)
        thresholds_to_be_removed_for_icustayid_itemid = thresholds_by_icustay_parameter_charttime[['ICUSTAY_ID', 'ITEMID_VALUE', 'CHARTTIME', 'CLEANING_FLAG']]

        thresholds_to_be_removed = thresholds_to_be_removed.append(thresholds_to_be_removed_for_icustayid_itemid, ignore_index=True)

display(thresholds_to_be_removed)

,ICUSTAY_ID,ITEMID_VALUE,CHARTTIME,CLEANING_FLAG
0,200075.0,220277,2159-09-23 01:35:00,Threshold removal due to overlap
1,200143.0,220045,2191-04-28 08:00:00,Threshold removal due to overlap
2,200153.0,220045,2194-08-16 19:55:00,Threshold removal due to overlap
3,200550.0,220045,2182-08-10 20:00:00,Threshold removal due to overlap
4,200571.0,220045,2150-12-19 17:24:00,Threshold removal due to overlap
...,...,...,...,...
769,299554.0,220179,2180-08-29 08:00:00,Threshold removal due to overlap
770,299725.0,220277,2172-09-18 20:45:00,Threshold removal due to overlap
771,299742.0,220277,2134-11-28 20:29:00,Threshold removal due to overlap
772,299847.0,220277,2200-11-11 20:13:00,Threshold removal due to overlap


In [7]:
chartevents_cleaning_06_overlapping_thresholds_removed = chartevents_cleaning_05_thresholds_in_valid_range.copy()

In [8]:
# Unnecessarily complicated step; could be simplified by better preparation of the data frame to be merged.
# Needed because the thresholds_to_be_removed data frame does contain the ITEMID_VALUE (i.e. the ITEMIDs of the vital parameters) but not the alarm threshold ITEMIDs.
import pandas as pd
import numpy as np

parameters = pd.DataFrame({
    'LABEL':            ['HR',      'NBPs',     'SpO2'],
    'VALUE':            [220045,    220179,     220277],
    'THRESHOLD_HIGH':   [220046,    223751,     223769],
    'THRESHOLD_LOW':    [220047,    223752,     223770]})

thresholds_to_be_removed.insert(loc=len(thresholds_to_be_removed.columns), column='ITEMID_THRESHOLD_HIGH', value=np.nan)
thresholds_to_be_removed.insert(loc=len(thresholds_to_be_removed.columns), column='ITEMID_THRESHOLD_LOW', value=np.nan)

for i, parameter in parameters.iterrows():

    thresholds_to_be_removed.loc[thresholds_to_be_removed.ITEMID_VALUE == parameter['VALUE'], 'ITEMID_THRESHOLD_HIGH'] = parameter['THRESHOLD_HIGH']
    thresholds_to_be_removed.loc[thresholds_to_be_removed.ITEMID_VALUE == parameter['VALUE'], 'ITEMID_THRESHOLD_LOW'] = parameter['THRESHOLD_LOW']

display(thresholds_to_be_removed)

,ICUSTAY_ID,ITEMID_VALUE,CHARTTIME,CLEANING_FLAG,ITEMID_THRESHOLD_HIGH,ITEMID_THRESHOLD_LOW
0,200075.0,220277,2159-09-23 01:35:00,Threshold removal due to overlap,223769.0,223770.0
1,200143.0,220045,2191-04-28 08:00:00,Threshold removal due to overlap,220046.0,220047.0
2,200153.0,220045,2194-08-16 19:55:00,Threshold removal due to overlap,220046.0,220047.0
3,200550.0,220045,2182-08-10 20:00:00,Threshold removal due to overlap,220046.0,220047.0
4,200571.0,220045,2150-12-19 17:24:00,Threshold removal due to overlap,220046.0,220047.0
...,...,...,...,...,...,...
769,299554.0,220179,2180-08-29 08:00:00,Threshold removal due to overlap,223751.0,223752.0
770,299725.0,220277,2172-09-18 20:45:00,Threshold removal due to overlap,223769.0,223770.0
771,299742.0,220277,2134-11-28 20:29:00,Threshold removal due to overlap,223769.0,223770.0
772,299847.0,220277,2200-11-11 20:13:00,Threshold removal due to overlap,223769.0,223770.0


In [9]:
thresholds_to_be_removed_high = thresholds_to_be_removed[
    ['ICUSTAY_ID','CHARTTIME','CLEANING_FLAG','ITEMID_THRESHOLD_HIGH']
    ].rename(columns = {'ITEMID_THRESHOLD_HIGH':'ITEMID'})
thresholds_to_be_removed_high = thresholds_to_be_removed_high[['ICUSTAY_ID','ITEMID','CHARTTIME','CLEANING_FLAG']]

thresholds_to_be_removed_low = thresholds_to_be_removed[
    ['ICUSTAY_ID','CHARTTIME','CLEANING_FLAG','ITEMID_THRESHOLD_LOW']
    ].rename(columns = {'ITEMID_THRESHOLD_LOW':'ITEMID'})
thresholds_to_be_removed_low = thresholds_to_be_removed_low[['ICUSTAY_ID','ITEMID','CHARTTIME','CLEANING_FLAG']]

In [12]:
# Takes quite long; there is probably a smarter way to insert/replace the threshold values in the VALUENUM_CLEAN column

for i, row in thresholds_to_be_removed_high.iterrows():

    chartevents_cleaning_06_overlapping_thresholds_removed.loc[
        (chartevents_cleaning_06_overlapping_thresholds_removed.ICUSTAY_ID == row['ICUSTAY_ID']) &
        (chartevents_cleaning_06_overlapping_thresholds_removed.ITEMID == row['ITEMID']) &
        (chartevents_cleaning_06_overlapping_thresholds_removed.CHARTTIME == row['CHARTTIME']), 
        'VALUENUM_CLEAN'] = None

    chartevents_cleaning_06_overlapping_thresholds_removed.loc[
        (chartevents_cleaning_06_overlapping_thresholds_removed.ICUSTAY_ID == row['ICUSTAY_ID']) &
        (chartevents_cleaning_06_overlapping_thresholds_removed.ITEMID == row['ITEMID']) &
        (chartevents_cleaning_06_overlapping_thresholds_removed.CHARTTIME == row['CHARTTIME']), 
        'CLEANING_FLAG'] = "Threshold removal due to overlap"

for i, row in thresholds_to_be_removed_low.iterrows():

    chartevents_cleaning_06_overlapping_thresholds_removed.loc[
        (chartevents_cleaning_06_overlapping_thresholds_removed.ICUSTAY_ID == row['ICUSTAY_ID']) &
        (chartevents_cleaning_06_overlapping_thresholds_removed.ITEMID == row['ITEMID']) &
        (chartevents_cleaning_06_overlapping_thresholds_removed.CHARTTIME == row['CHARTTIME']), 
        'VALUENUM_CLEAN'] = None

    chartevents_cleaning_06_overlapping_thresholds_removed.loc[
        (chartevents_cleaning_06_overlapping_thresholds_removed.ICUSTAY_ID == row['ICUSTAY_ID']) &
        (chartevents_cleaning_06_overlapping_thresholds_removed.ITEMID == row['ITEMID']) &
        (chartevents_cleaning_06_overlapping_thresholds_removed.CHARTTIME == row['CHARTTIME']), 
        'CLEANING_FLAG'] = "Threshold removal due to overlap"

display(chartevents_cleaning_06_overlapping_thresholds_removed[
    chartevents_cleaning_06_overlapping_thresholds_removed.CLEANING_FLAG == "Threshold removal due to overlap"])

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUENUM,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED,VALUENUM_CLEAN,CLEANING_FLAG
6265,19732601.0,67800.0,132255.0,200075.0,223769.0,2159-09-23 01:35:00,2159-09-23 01:35:00,20030.0,10,10.0,%,0.0,0.0,None,None,NaN,Threshold removal due to overlap
6270,19732602.0,67800.0,132255.0,200075.0,223770.0,2159-09-23 01:35:00,2159-09-23 01:35:00,20030.0,90,90.0,%,0.0,0.0,None,None,NaN,Threshold removal due to overlap
11593,31970192.0,94414.0,170244.0,200143.0,220046.0,2191-04-28 08:00:00,2191-04-28 10:56:00,17140.0,50,50.0,bpm,0.0,0.0,None,None,NaN,Threshold removal due to overlap
11659,31970193.0,94414.0,170244.0,200143.0,220047.0,2191-04-28 08:00:00,2191-04-28 10:56:00,17140.0,120,120.0,bpm,0.0,0.0,None,None,NaN,Threshold removal due to overlap
12008,20740822.0,69995.0,164810.0,200153.0,220046.0,2194-08-16 19:55:00,2194-08-16 19:57:00,14772.0,60,60.0,bpm,0.0,0.0,None,None,NaN,Threshold removal due to overlap
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7862151,4154449.0,29660.0,186040.0,299742.0,223770.0,2134-11-28 20:29:00,2134-11-28 20:30:00,20053.0,85,85.0,%,0.0,0.0,None,None,NaN,Threshold removal due to overlap
7868322,8783149.0,46439.0,132850.0,299847.0,223769.0,2200-11-11 20:13:00,2200-11-11 20:14:00,17446.0,8,8.0,%,0.0,0.0,None,None,NaN,Threshold removal due to overlap
7868329,8783150.0,46439.0,132850.0,299847.0,223770.0,2200-11-11 20:13:00,2200-11-11 20:14:00,17446.0,85,85.0,%,0.0,0.0,None,None,NaN,Threshold removal due to overlap
7872449,12444451.0,54358.0,134569.0,299889.0,223769.0,2172-06-04 07:55:00,2172-06-04 07:55:00,15659.0,88,88.0,%,0.0,0.0,None,None,NaN,Threshold removal due to overlap


In [14]:
import pandas as pd
import pyarrow as pa
path_to_dir_chartevents_clean = './reportdata/'
# Save chartevents_cleaning_06_overlapping_thresholds_removed as parquet file
chartevents_cleaning_06_overlapping_thresholds_removed.to_parquet(path_to_dir_chartevents_clean+'chartevents_cleaning_06_overlapping_thresholds_removed.parquet', engine='pyarrow')

## Save final chartevents_clean data frame

In [15]:
import pandas as pd
import pyarrow as pa
# For now, the data frame is stored twice with different names, because it may be that further cleanup steps follow later, so that chartevents_cleaning_06_overlapping_thresholds_removed is not the final chartevents_clean anymore.
chartevents_clean = chartevents_cleaning_06_overlapping_thresholds_removed
# Save chartevents_clean as parquet file
path_to_dir_chartevents_clean = './reportdata/'
chartevents_clean.to_parquet(path_to_dir_chartevents_clean+'chartevents_clean.parquet', engine='pyarrow')